In [1]:
import tensorflow.compat.v2 as tf
import numpy as np

from tensorflow_probability.python.internal import dtype_util
from tensorflow_probability.python.distributions import distribution
from tensorflow_probability.python.internal import parameter_properties
from tensorflow_probability.python.internal import tensor_util
from scipy import fft
from math import pi

2023-04-14 09:38:29.819201: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-14 09:38:29.900298: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-14 09:38:29.901798: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-14 09:38:31.514592: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
import tensorflow_probability as tfp
i = [0.91343856, 0.03249955, 0.12706113]
a = np.array(tfp.distributions.Binomial(total_count = 1,probs = i[2]).sample(1))[0]

Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.while_loop(c, b, vars, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.while_loop(c, b, vars))


2023-04-14 09:38:33.762820: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'shape' with dtype int32 and shape [1]
	 [[{{node shape}}]]
2023-04-14 09:38:33.796774: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'binomial_noncpu/while/uniform/stateless_random_uniform/StatelessRandomUniformV2/binomial_noncpu/concat' with dtype int32 and shape [1]
	 [[{{node binomial_noncpu/while/uniform/stateless_random_uniform/StatelessRandomUniformV2/binomial_noncpu/concat}}]]
2023-04-14 09:38:33.796945: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an err

In [3]:
def samples_n(n,probs,seed=None):
   res = np.zeros(n)
   i = 0
   for j in probs:
    for i in range(n):
        res[i] += np.array(tfp.distributions.Binomial(total_count = 1,probs = j).sample(1))[0]
   return(res)

In [ ]:
np.array(tfp.distributions.Binomial(total_count = 1,probs = i[0]).sample(1))[0]

1.0

In [4]:
i = np.array(tfp.distributions.Uniform().sample(10))
s = samples_n(10,i)

In [5]:
k  = [0.14971250, 0.89603616, 0.46497399, 0.08322113, 0.10419799, 0.92368582, 0.47016735, 0.65425167, 0.89811465, 0.07596209]

In [6]:
samps = samples_n(10,k)

## Roadmap
### PMF 

1.   dpb_conv : PMF using Convolutions
2.   dpb_dc : PMF using Direct Convolution
3.   dpb_dftcf : PMF using DC-FFT

### CDF 

1.   ppb_conv : CDF using Convolutions
2.   ppb_dc : CDF using Direct Convolution
3.   ppb_dftcf : CDF using DC-FFT


### samples

Generate Samples for the distribution




In [23]:
k

[0.1497125,
 0.89603616,
 0.46497399,
 0.08322113,
 0.10419799,
 0.92368582,
 0.47016735,
 0.65425167,
 0.89811465,
 0.07596209]

In [22]:
samps

array([6., 5., 3., 3., 4., 4., 3., 4., 5., 4.])

In [45]:
dpb_conv(samps,k)

array([8.50287500e-01, 9.00113923e-06, 3.01317108e-04, 3.67681750e-03,
       1.98512766e-02, 4.67064051e-02, 5.00952125e-02, 2.41270199e-02,
       4.56909320e-03, 3.65872257e-04, 1.04846872e-05])

In [38]:
type(samps[0])

numpy.float64

In [43]:
k = np.float64(k)

In [7]:
def ppb_generic(pmf):
  return np.cumsum(pmf)

## **Convolution** 

In [10]:
def dpb_conv(obs,probs):
  size = len(probs)
  res = np.zeros(size+1)
  res[0] = 1-probs[0]
  res[1] = probs[0]
  for i in range(1,size):
    if(probs[i]):
      for j in range(i,-1,-1):
        if (res[j]):
          res[j+1] += res[j]*probs[i]
          res[j] *= 1 - probs[i]
  res /= np.sum(res)
  return res

In [11]:
def ppb_conv(obs,probs):
  size = probs(len)
  pmf = dpb_conv(obs,probs)
  res = ppb_generic(pmf)
  return (res)

## **Direct Conv**


In [14]:
def dpb_conv_2(obs, probs):
    # number of input probabilities
    size = len(probs)
    
    # results vector
    results = np.zeros(size + 1)
    results[0] = 1 - probs[0]
    results[1] = probs[0]
    
    for i in range(1, size):
        # check for user interrupt
        # not available in Python
        
        if probs[i]:
            for j in range(i, -1, -1):
                if results[j]:
                    results[j + 1] += results[j] * probs[i]
                    results[j] *= 1 - probs[i]
    
    # make sure that probability masses sum up to 1
    results /= np.sum(results)
    
    # return final results
    return results

In [15]:
dpb_conv_2(samps,k)

array([5.11216911e-05, 1.72032231e-03, 2.11836881e-02, 1.16421527e-01,
       2.85118856e-01, 3.31220611e-01, 1.87123874e-01, 5.00770430e-02,
       6.64705333e-03, 4.25419713e-04, 1.04846872e-05])

In [16]:
dpb_conv(samps,k)

array([5.11216911e-05, 1.72032231e-03, 2.11836881e-02, 1.16421527e-01,
       2.85118856e-01, 3.31220611e-01, 1.87123874e-01, 5.00770430e-02,
       6.64705333e-03, 4.25419713e-04, 1.04846872e-05])

In [3]:
!pip install pyfftw

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 35.9 MB/s eta 0:00:00:00:01


In [9]:
import pyfftw
a = pyfftw.empty_aligned(2,dtype='float64')

In [11]:
a.imag

array([0., 0.])

In [8]:
type(pyfftw.zeros_aligned(3))

numpy.ndarray

In [12]:
def fft_probs(probs_A, probs_B):
    sizeA = len(probsA)
    sizeB = len(probsB)
    sizeRes = sizeA + sizeB -1

    res = np.empty(sizeRes,dtype = 'float64')

    probsA_fft = pyfftw.empty_aligned(sizeRes)
    probsB_fft = pyfftw.empty_aligned(sizeRes)
    result_fft = pyfftw.empty_aligned(sizeRes)

    padded_probsA = pyfftw.zeros_aligned(sizeRes)
    padded_probsA[:sizeA] = probs_A
    fftw_planA = pyfftw(padded_probsA,probsA_fft,flags=('FFTW_ESTIMATE', 'FFTW_DESTROY_INPUT'))
    fftw_planA.execute()

    padded_probsB = pyfftw.zeros_aligned(sizeRes)
    padded_probsB[:sizeB] = probs_B
    fftw_planB = pyfftw(padded_probsB,probsB_fft,flags=('FFTW_ESTIMATE', 'FFTW_DESTROY_INPUT'))
    fftw_planB.execute()

    for i in range(sizeRes):
        result_fft[i].real = (probsA_fft[i].real*probsB_fft[i].real - probsA_fft[i].imag*probsB_fft[i].imag)/sizeRes
        result_fft[i].imag = (probsA_fft[i].real*probsB_fft[i].imag + probsA_fft[i].imag*probsB_fft[i].real)/sizeRes

    planResult = pyfftw.FFTW(result_fft, res, direction='FFTW_BACKWARD', flags=('FFTW_ESTIMATE', 'FFTW_DESTROY_INPUT'))
    planResult.execute()

    return (res)




   


In [ ]:
def dpb_dc(obs,probs):
    size = len(probs)
    num_splits = if size > 1950